<a href="https://colab.research.google.com/github/ekalmes/Portfolio/blob/main/SwangoMetrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregating Financial Data for Areas Surrounding Zip Codes, and Mapping Them to Closest Airport 

Written by: Zeke Kalmes, for Bret Swango, Head of Workforce and Site Selection Analytics, Colliers International

## Imports

In [ ]:
#general imports
#!pip install geopandas
#!pip install geopy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
import random

# import classifiers
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# import metrics for assessing classification models
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
# imports and functions for authenticating google account
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#id of zip file in google drive
file_id = '1kJvSzyeZMzrL85kDbgFQ_ZTpcPnW05-2'
downloaded = drive.CreateFile({'id': file_id})

In [ ]:
#user must download file into notebook 
downloaded.GetContentFile('colliers_analysis_7.zip')

In [ ]:
!ls -lha colliers_analysis_7.zip

-rw-r--r-- 1 root root 453M Jul 29 03:43 colliers_analysis_7.zip


In [ ]:
#unzipping images folder
filelist = !unzip colliers_analysis_7.zip

## Data Cleaning

### Amazon Warehouses

In [ ]:
df_amazon = pd.read_csv('colliers_analysis/re_amazon_distribution copy.csv', 
                        usecols=[1,3,4,5,6,7,8,9])
df_amazon.dropna(inplace=True)
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 764 entries, 0 to 828
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   brad_address_raw               764 non-null    object
 1   brad_src_file_nm               764 non-null    object
 2   brad_square_footage            764 non-null    object
 3   brad_year_opened               764 non-null    object
 4   brad_description_of_operation  764 non-null    object
 5   brad_state_nm                  764 non-null    object
 6   brad_location_type             764 non-null    object
 7   brad_process_ts                764 non-null    object
dtypes: object(8)
memory usage: 53.7+ KB


### Cargo

In [ ]:
df_cargo = pd.read_csv('colliers_analysis/Cargo Volume.xlsx - Sheet2.csv')
df_cargo.rename(columns={"zip":"ZIP"}, inplace = True)
df_cargo.drop(columns=['Unnamed: 0'], inplace = True)
df_cargo.head()

,airport_name,IATA_code,city,state,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,10_year_per_chg,14_'19_per_chg,10_year_rank,14_'19_rank,address,ZIP,latitude,longitude
0,Memphis International Airport,MEM,Memphis,TN,"24,344,504,836","24,432,753,510","23,949,525,780","23,866,469,898","22,679,195,919","22,774,592,279","21,891,425,638","20,983,699,672","20,303,149,106","19,544,635,833","18,928,729,202","19,500,093,674",24.6%,6.9%,19,30,2491 Winchester Rd,38116,35.040031,-89.981873
1,Ted Stevens Anchorage International Airport,ANC,Anchorage,AK,"18,306,699,196","18,413,943,946","17,337,337,377","16,867,292,945","17,139,250,601","15,867,941,046","15,982,410,652","16,522,948,234","17,774,071,223","19,463,543,790","15,524,360,013","17,951,597,580",-5.9%,15.4%,26,26,5000 W International Airport Rd,99502,61.175900,-149.990100
2,Louisville International Airport,SDF,Louisville,KY,"15,599,137,404","14,643,310,355","13,403,682,652","12,804,148,855","12,057,543,654","11,568,369,154","21,891,425,638","20,983,699,672","20,303,149,106","19,544,635,833","18,928,729,202","19,500,093,674",-20.2%,34.8%,29,17,600 Terminal Dr,40209,38.175662,-85.736923
3,Miami International Airport,MIA,Miami,FL,"9,235,113,239","8,398,363,905","7,963,988,407","7,899,307,235","7,630,761,702","7,192,790,882","6,847,177,300","7,147,983,325","6,634,448,852","6,905,291,871","6,352,786,009","6,988,513,672",33.7%,28.4%,13,20,5600 NW 36th St,33166,25.795865,-80.287046
4,Los Angeles International Airport,LAX,Los Angeles,CA,"7,459,422,818","7,316,551,753","7,197,930,264","6,931,158,178","6,585,460,219","4,297,359,912","4,199,375,809","4,204,996,790","4,043,122,100","3,954,810,091","3,768,864,700","5,751,595,501",88.6%,73.6%,2,7,1 World Way,90045,33.941589,-118.408530


In [ ]:
df_cargo.drop(df_cargo[df_cargo['state'] == 'PR'].index, inplace = True)
df_cargo.reset_index(drop=True, inplace=True)
df_cargo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airport_name     48 non-null     object 
 1   IATA_code        48 non-null     object 
 2   city             48 non-null     object 
 3   state            48 non-null     object 
 4   2019             48 non-null     object 
 5   2018             48 non-null     object 
 6   2017             48 non-null     object 
 7   2016             48 non-null     object 
 8   2015             48 non-null     object 
 9   2014             48 non-null     object 
 10  2013             48 non-null     object 
 11  2012             48 non-null     object 
 12  2011             48 non-null     object 
 13  2010             48 non-null     object 
 14  2009             48 non-null     object 
 15  2008             48 non-null     object 
 16  10_year_per_chg  48 non-null     object 
 17  14_'19_per_chg   4

In [ ]:
df_cargo.set_index('ZIP', inplace = True)

### Labor

In [ ]:
df_labor = pd.read_csv('colliers_analysis/Occupation_Map_Wages_16_63_or_Less_in_2222_ZIPs_809.xlsx - Occupations.csv',
                       usecols=[0,1,2,3,4,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])
df_labor.drop(df_labor[df_labor['2020 Net Commuters'] == 'Insf. Data'].index, inplace = True)
df_labor.reset_index(drop=True, inplace=True)
df_labor.rename(columns={"ZIP Name":"ZIP_Name"}, inplace = True)
df_labor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34453 entries, 0 to 34452
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ZIP                                  34453 non-null  int64  
 1   ZIP_Name                             34453 non-null  object 
 2   2019 Jobs                            34453 non-null  object 
 3   2020 Jobs                            34453 non-null  object 
 4   2019 - 2020 Change                   34453 non-null  object 
 5   Regional Completions (2019)          34453 non-null  float64
 6   Automation Index                     34453 non-null  float64
 7   2020 Resident Workers                34453 non-null  object 
 8   2020 Net Commuters                   34453 non-null  object 
 9   Avg. Hourly Earnings                 34453 non-null  object 
 10  Median Hourly Earnings               34453 non-null  object 
 11  Median Annual Earnings      

In [ ]:
df_labor['2020 Net Commuters'] = pd.to_numeric(df_labor['2020 Net Commuters'], errors = 'coerce')
df_labor['2020 Jobs'] = pd.to_numeric(df_labor['2020 Jobs'], errors = 'coerce')
df_labor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34453 entries, 0 to 34452
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ZIP                                  34453 non-null  int64  
 1   ZIP_Name                             34453 non-null  object 
 2   2019 Jobs                            34453 non-null  object 
 3   2020 Jobs                            34453 non-null  int64  
 4   2019 - 2020 Change                   34453 non-null  object 
 5   Regional Completions (2019)          34453 non-null  float64
 6   Automation Index                     34453 non-null  float64
 7   2020 Resident Workers                34453 non-null  object 
 8   2020 Net Commuters                   34453 non-null  float64
 9   Avg. Hourly Earnings                 34453 non-null  object 
 10  Median Hourly Earnings               34453 non-null  object 
 11  Median Annual Earnings      

### 15 min

In [ ]:
df_dt_15 = pd.read_csv('colliers_analysis/drivetime_zip_intersect_15 copy.csv',
                       usecols=[1,2,3,4,8,9,10])
# Renaming columns to portray distance so drivetime column can be dropped
df_dt_15.rename(columns = {"sdzi_intersect_zip_cd":"sdzi_15_intersect_zip_cd", 
                           "sdzi_intersect_zip_area":"sdzi_15_intersect_zip_area", 
                           "sdzi_intersect_area":"sdzi_15_intersect_area",
                           "sdzi_zip_cd":"ZIP"}, inplace = True)
df_dt_15['sdzi_latitude'] = pd.to_numeric(df_dt_15['sdzi_latitude'], errors='coerce')
df_dt_15['sdzi_longitude'] = pd.to_numeric(df_dt_15['sdzi_longitude'], errors = 'coerce')
df_dt_15['ZIP'] = pd.to_numeric(df_dt_15['ZIP'], errors='coerce')
df_dt_15['sdzi_15_intersect_zip_cd'] = pd.to_numeric(df_dt_15['sdzi_15_intersect_zip_cd'], errors = 'coerce')
df_dt_15['sdzi_15_intersect_zip_area'] = pd.to_numeric(df_dt_15['sdzi_15_intersect_zip_area'], errors='coerce')
df_dt_15['sdzi_15_intersect_area'] = pd.to_numeric(df_dt_15['sdzi_15_intersect_area'], errors = 'coerce')
print(df_dt_15.info())
print(df_dt_15['sdzi_latitude'][20])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257436 entries, 0 to 257435
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   sdzi_zip_nm                 257436 non-null  object 
 1   ZIP                         257435 non-null  float64
 2   sdzi_longitude              257435 non-null  float64
 3   sdzi_latitude               257435 non-null  float64
 4   sdzi_15_intersect_zip_cd    257435 non-null  float64
 5   sdzi_15_intersect_zip_area  257435 non-null  float64
 6   sdzi_15_intersect_area      257435 non-null  float64
dtypes: float64(6), object(1)
memory usage: 13.7+ MB
None
36.245629770177594


In [ ]:
# MultiIndex Using sdzi_zip_cd as 0 level, sdzi_15_intersect_zip_cd as level 1
df_dt_15.set_index(['ZIP', 'sdzi_15_intersect_zip_cd'], inplace = True)
df_dt_15.sort_index(inplace = True)
print(df_dt_15.loc[[27]])

                                  sdzi_zip_nm  ... sdzi_15_intersect_area
ZIP sdzi_15_intersect_zip_cd                   ...                       
27  27                        Pike Ntl Forest  ...                 2.5569
    80809                     Pike Ntl Forest  ...                5.53111
    80829                     Pike Ntl Forest  ...              0.0670102

[3 rows x 5 columns]


### 30 min

In [ ]:
df_dt_30 = pd.read_csv('colliers_analysis/drivetime_zip_intersect_30 copy.csv',
                       usecols=[1,2,3,4,8,9,10])
df_dt_30.rename(columns = {"sdzi_intersect_zip_cd":"ZIP", 
                           "sdzi_intersect_zip_area":"sdzi_30_intersect_zip_area", 
                           "sdzi_intersect_area":"sdzi_30_intersect_area"}, inplace = True)
df_dt_30['sdzi_latitude'] = pd.to_numeric(df_dt_30['sdzi_latitude'], errors='coerce')
df_dt_30['sdzi_longitude'] = pd.to_numeric(df_dt_30['sdzi_longitude'], errors = 'coerce')
df_dt_30['ZIP'] = pd.to_numeric(df_dt_30['ZIP'], errors='coerce')
df_dt_30['sdzi_zip_cd'] = pd.to_numeric(df_dt_30['sdzi_zip_cd'], errors = 'coerce')
df_dt_30['sdzi_30_intersect_zip_area'] = pd.to_numeric(df_dt_30['sdzi_30_intersect_zip_area'], errors='coerce')
df_dt_30['sdzi_30_intersect_area'] = pd.to_numeric(df_dt_30['sdzi_30_intersect_area'], errors = 'coerce')
print(df_dt_30.info())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861804 entries, 0 to 861803
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   sdzi_zip_nm                 861804 non-null  object 
 1   sdzi_zip_cd                 861799 non-null  float64
 2   sdzi_longitude              861799 non-null  float64
 3   sdzi_latitude               861799 non-null  float64
 4   ZIP                         861799 non-null  float64
 5   sdzi_30_intersect_zip_area  861799 non-null  float64
 6   sdzi_30_intersect_area      861799 non-null  float64
dtypes: float64(6), object(1)
memory usage: 46.0+ MB
None


In [ ]:
#df_dt_30.set_index('ZIP', inplace=True)

### 45 min

In [ ]:
df_dt_45 = pd.read_csv('colliers_analysis/drivetime_zip_intersect_45 copy.csv',
                       usecols=[1,2,3,4,8,9,10])
df_dt_45.rename(columns = {"sdzi_intersect_zip_cd":"sdzi_45_intersect_zip_cd", 
                           "sdzi_intersect_zip_area":"sdzi_45_intersect_zip_area", 
                           "sdzi_intersect_area":"sdzi_45_intersect_area",
                           "sdzi_zip_cd":"ZIP"}, inplace = True)
df_dt_45['sdzi_latitude'] = pd.to_numeric(df_dt_45['sdzi_latitude'], errors='coerce')
df_dt_45['sdzi_longitude'] = pd.to_numeric(df_dt_45['sdzi_longitude'], errors = 'coerce')
df_dt_45['ZIP'] = pd.to_numeric(df_dt_45['ZIP'], errors='coerce')
df_dt_45['sdzi_45_intersect_zip_cd'] = pd.to_numeric(df_dt_45['sdzi_45_intersect_zip_cd'], errors = 'coerce')
df_dt_45['sdzi_45_intersect_zip_area'] = pd.to_numeric(df_dt_45['sdzi_45_intersect_zip_area'], errors='coerce')
df_dt_45['sdzi_45_intersect_area'] = pd.to_numeric(df_dt_45['sdzi_45_intersect_area'], errors = 'coerce')
print(df_dt_45.info())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790220 entries, 0 to 1790219
Data columns (total 7 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   sdzi_zip_nm                 object 
 1   ZIP                         float64
 2   sdzi_longitude              float64
 3   sdzi_latitude               float64
 4   sdzi_45_intersect_zip_cd    float64
 5   sdzi_45_intersect_zip_area  float64
 6   sdzi_45_intersect_area      float64
dtypes: float64(6), object(1)
memory usage: 95.6+ MB
None


In [ ]:
df_dt_45.set_index(['ZIP', 'sdzi_45_intersect_zip_cd'], inplace = True)
df_dt_45.sort_index(inplace = True)
print(df_dt_45.loc[[27]])

                                  sdzi_zip_nm  ... sdzi_45_intersect_area
ZIP sdzi_45_intersect_zip_cd                   ...                       
27  27                        Pike Ntl Forest  ...                 10.573
    80132                     Pike Ntl Forest  ...                2.09027
    80135                     Pike Ntl Forest  ...               0.549064
    80809                     Pike Ntl Forest  ...                21.3645
    80814                     Pike Ntl Forest  ...                16.2855
    80816                     Pike Ntl Forest  ...                4.41687
    80817                     Pike Ntl Forest  ...                14.1824
    80819                     Pike Ntl Forest  ...                4.30384
    80829                     Pike Ntl Forest  ...                9.73276
    80840                     Pike Ntl Forest  ...                5.91199
    80863                     Pike Ntl Forest  ...                45.4359
    80902                     Pike Ntl

### 60 min

In [ ]:
df_dt_60 = pd.read_csv('colliers_analysis/drivetime_zip_intersect_60 copy.csv',
                       usecols=[1,2,3,4,8,9,10])
df_dt_60.rename(columns = {"sdzi_intersect_zip_cd":"sdzi_60_intersect_zip_cd", 
                           "sdzi_intersect_zip_area":"sdzi_60_intersect_zip_area", 
                           "sdzi_intersect_area":"sdzi_60_intersect_area",
                           "sdzi_zip_cd":"ZIP"}, inplace = True)
df_dt_60['sdzi_latitude'] = pd.to_numeric(df_dt_60['sdzi_latitude'], errors='coerce')
df_dt_60['sdzi_longitude'] = pd.to_numeric(df_dt_60['sdzi_longitude'], errors = 'coerce')
df_dt_60['ZIP'] = pd.to_numeric(df_dt_60['ZIP'], errors='coerce')
df_dt_60['sdzi_60_intersect_zip_cd'] = pd.to_numeric(df_dt_60['sdzi_60_intersect_zip_cd'], errors = 'coerce')
df_dt_60['sdzi_60_intersect_zip_area'] = pd.to_numeric(df_dt_60['sdzi_60_intersect_zip_area'], errors='coerce')
df_dt_60['sdzi_60_intersect_area'] = pd.to_numeric(df_dt_60['sdzi_60_intersect_area'], errors = 'coerce')
print(df_dt_60.info())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997894 entries, 0 to 2997893
Data columns (total 7 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   sdzi_zip_nm                 object 
 1   ZIP                         float64
 2   sdzi_longitude              float64
 3   sdzi_latitude               float64
 4   sdzi_60_intersect_zip_cd    float64
 5   sdzi_60_intersect_zip_area  float64
 6   sdzi_60_intersect_area      float64
dtypes: float64(6), object(1)
memory usage: 160.1+ MB
None


In [ ]:
df_dt_60.set_index(['ZIP', 'sdzi_60_intersect_zip_cd'], inplace = True)
df_dt_60.sort_index(inplace = True)
print(df_dt_60.loc[[27]])

                                  sdzi_zip_nm  ... sdzi_60_intersect_area
ZIP sdzi_60_intersect_zip_cd                   ...                       
27  27                        Pike Ntl Forest  ...                17.0559
    80104                     Pike Ntl Forest  ...                1.97133
    80106                     Pike Ntl Forest  ...              0.0573367
    80109                     Pike Ntl Forest  ...               0.358221
    80116                     Pike Ntl Forest  ...               0.699242
    80118                     Pike Ntl Forest  ...                21.6981
    80132                     Pike Ntl Forest  ...                32.6893
    80133                     Pike Ntl Forest  ...                2.41321
    80135                     Pike Ntl Forest  ...                13.1268
    80808                     Pike Ntl Forest  ...               0.647872
    80809                     Pike Ntl Forest  ...                23.9285
    80813                     Pike Ntl

### 120 min

In [ ]:
df_dt_120 = pd.read_csv('colliers_analysis/drivetime_zip_intersect_120 copy.csv',
                        usecols=[1,2,3,4,8,9,10])
df_dt_120.rename(columns = {"sdzi_intersect_zip_cd":"ZIP", 
                           "sdzi_intersect_zip_area":"sdzi_120_intersect_zip_area", 
                           "sdzi_intersect_area":"sdzi_120_intersect_area"}, inplace = True)
df_dt_120['sdzi_latitude'] = pd.to_numeric(df_dt_120['sdzi_latitude'], errors='coerce')
df_dt_120['sdzi_longitude'] = pd.to_numeric(df_dt_120['sdzi_longitude'], errors = 'coerce')
df_dt_120['sdzi_zip_cd'] = pd.to_numeric(df_dt_120['sdzi_zip_cd'], errors='coerce')
df_dt_120['ZIP'] = pd.to_numeric(df_dt_120['ZIP'], errors = 'coerce')
df_dt_120['sdzi_120_intersect_zip_area'] = pd.to_numeric(df_dt_120['sdzi_120_intersect_zip_area'], errors='coerce')
df_dt_120['sdzi_120_intersect_area'] = pd.to_numeric(df_dt_120['sdzi_120_intersect_area'], errors = 'coerce')
print(df_dt_120.info())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13143722 entries, 0 to 13143721
Data columns (total 7 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   sdzi_zip_nm                  object 
 1   sdzi_zip_cd                  float64
 2   sdzi_longitude               float64
 3   sdzi_latitude                float64
 4   ZIP                          float64
 5   sdzi_120_intersect_zip_area  float64
 6   sdzi_120_intersect_area      float64
dtypes: float64(6), object(1)
memory usage: 702.0+ MB
None


In [ ]:
#df_dt_120.set_index(['ZIP', 'sdzi_120_intersect_zip_cd'], inplace = True)
#df_dt_120.sort_index(inplace = True)
#print(df_dt_120.loc[[27]])

                                   sdzi_zip_nm  ... sdzi_120_intersect_area
ZIP sdzi_120_intersect_zip_cd                   ...                        
27  27                         Pike Ntl Forest  ...                 42.9875
    80002                      Pike Ntl Forest  ...                 5.71565
    80003                      Pike Ntl Forest  ...                 6.53717
    80004                      Pike Ntl Forest  ...                 7.80868
    80005                      Pike Ntl Forest  ...                 11.7197
...                                        ...  ...                     ...
    81236                      Pike Ntl Forest  ...                 23.5395
    81240                      Pike Ntl Forest  ...                 222.989
    81244                      Pike Ntl Forest  ...                 1.79359
    81252                      Pike Ntl Forest  ...                 33.2442
    81253                      Pike Ntl Forest  ...                  95.527

[243 rows x

## Joining Labor and Distance Data on MultiIndex

### Joining Labor and 120 intersect zip codes 

This will map the intersect zip codes that have labor data to the labor data zip codes. Then, this new df will be mapped to each ranges original zip codes, and will be able to aggregate labor data of the surrounding zip codes. 

In [ ]:
df_map_120_zc = df_labor.merge(df_dt_120, how='inner')
print(df_map_120_zc.info())
print(df_map_120_zc.head(100))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12270042 entries, 0 to 12270041
Data columns (total 28 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   ZIP                                  int64  
 1   ZIP_Name                             object 
 2   2019 Jobs                            object 
 3   2020 Jobs                            object 
 4   2019 - 2020 Change                   object 
 5   Regional Completions (2019)          float64
 6   Automation Index                     float64
 7   2020 Resident Workers                object 
 8   2020 Net Commuters                   float64
 9   Avg. Hourly Earnings                 object 
 10  Median Hourly Earnings               object 
 11  Median Annual Earnings               object 
 12  COL Index                            object 
 13  COL Adjusted Avg. Hourly Earnings    object 
 14  COL Adjusted Median Hourly Earnings  object 
 15  COL Adjusted Median Annual Ear

In [ ]:
df_map_15_zc = df_labor.merge(df_dt_15, how='inner', left_on='ZIP', right_on='sdzi_15_intersect_zip_cd')
print(df_map_15_zc.info())
print(df_map_15_zc.head(100))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245161 entries, 0 to 245160
Data columns (total 29 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ZIP_x                                245161 non-null  int64  
 1   ZIP_Name                             245161 non-null  object 
 2   2019 Jobs                            245161 non-null  object 
 3   2020 Jobs                            245161 non-null  object 
 4   2019 - 2020 Change                   245161 non-null  object 
 5   Regional Completions (2019)          245161 non-null  float64
 6   Automation Index                     245161 non-null  float64
 7   2020 Resident Workers                245161 non-null  object 
 8   2020 Net Commuters                   245161 non-null  float64
 9   Avg. Hourly Earnings                 245161 non-null  object 
 10  Median Hourly Earnings               245161 non-null  object 
 11  Median Annual

In [ ]:
# df_map_15_zc.set_index(['ZIP_y', 'sdzi_15_intersect_zip_cd'], inplace = True)
# df_map_15_zc.sort_index(inplace = True)
df_map_15_zc.head(100)

ZIP_x  ... sdzi_15_intersect_area
ZIP_y  sdzi_15_intersect_zip_cd         ...                       
12.0   96130.0                   96130  ...               0.045753
27.0   80809.0                   80809  ...               5.531113
       80829.0                   80829  ...               0.067010
29.0   95922.0                   95922  ...               0.743908
       95981.0                   95981  ...               0.005655
...                                ...  ...                    ...
1005.0 1543.0                     1543  ...               0.316637
1007.0 1002.0                     1002  ...               1.696739
       1007.0                     1007  ...              38.091411
       1009.0                     1009  ...               0.012042
       1033.0                     1033  ...               4.311263

[100 rows x 27 columns]

In [ ]:
df_map_15_zc.to_csv(r'colliers_analysis\15_min_labor_data.csv')

### 15 min

In [ ]:
df_labor_15 = df_labor.join(df_dt_15, how='inner', on='ZIP')
print(df_labor_15.head(100))
print(df_labor_15.info())

ValueError: ignored

### 30 min

In [ ]:
df_map_30_zc = df_labor.merge(df_dt_30, how='inner', on='ZIP')
print(df_map_30_zc.info())
print(df_map_30_zc.head(100))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 823362 entries, 0 to 823361
Data columns (total 28 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ZIP                                  823362 non-null  int64  
 1   ZIP_Name                             823362 non-null  object 
 2   2019 Jobs                            823362 non-null  object 
 3   2020 Jobs                            823362 non-null  int64  
 4   2019 - 2020 Change                   823362 non-null  object 
 5   Regional Completions (2019)          823362 non-null  float64
 6   Automation Index                     823362 non-null  float64
 7   2020 Resident Workers                823362 non-null  object 
 8   2020 Net Commuters                   823362 non-null  float64
 9   Avg. Hourly Earnings                 823362 non-null  object 
 10  Median Hourly Earnings               823362 non-null  object 
 11  Median Annual

In [ ]:
df_map_30_zc.set_index(['sdzi_zip_cd', 'ZIP'], inplace=True)
df_map_30_zc.sort_index(inplace=True)
df_map_30_zc.head(100)

ZIP_Name  ... sdzi_30_intersect_area
sdzi_zip_cd ZIP                   ...                       
12.0        95947     Greenville  ...               0.132932
            96130     Susanville  ...               0.876621
16.0        93518       Caliente  ...               0.026775
18.0        93015       Fillmore  ...               0.457163
22.0        95437     Fort Bragg  ...               0.093092
...                          ...  ...                    ...
164.0       56727         Grygla  ...               3.271102
168.0       3581          Gorham  ...               0.009739
            3813   Center Conway  ...               0.393560
172.0       3251         Lincoln  ...               0.034074
            3574       Bethlehem  ...               0.196659

[100 rows x 26 columns]

### 45 min

In [ ]:
df_labor_45 = df_labor.join(df_dt_45, how='inner')
print(df_labor_45.head(100))
print(df_labor_45.info())

                                                             ZIP_Name  ... sdzi_45_intersect_area_pct
ZIP  sdzi_45_intersect_zip_cd                                          ...                           
116  116                               [Idaho, ZIP Code not reported]  ...                   0.101074
119  119                                [Iowa, ZIP Code not reported]  ...                   0.563632
121  121                            [Kentucky, ZIP Code not reported]  ...                  0.0748723
125  125                       [Massachusetts, ZIP Code not reported]  ...                 0.00717913
126  126                            [Michigan, ZIP Code not reported]  ...                   0.152415
...                                                               ...  ...                        ...
1001 1108                                                      Agawam  ...                          1
     1109                                                      Agawam  ...        

### 60 min

In [ ]:
df_labor_60 = df_labor.join(df_dt_60, how='inner')
print(df_labor_60.head(100))
print(df_labor_60.info())

                                                             ZIP_Name  ... sdzi_60_intersect_area_pct
ZIP  sdzi_60_intersect_zip_cd                                          ...                           
116  116                               [Idaho, ZIP Code not reported]  ...                   0.179688
119  119                                [Iowa, ZIP Code not reported]  ...                   0.646881
     70357                              [Iowa, ZIP Code not reported]  ...                  0.0467614
121  121                            [Kentucky, ZIP Code not reported]  ...                   0.130413
125  125                       [Massachusetts, ZIP Code not reported]  ...                   0.010917
...                                                               ...  ...                        ...
1001 1033                                                      Agawam  ...                          1
     1034                                                      Agawam  ...        

### 120 min

In [ ]:
df_labor_120 = df_labor.merge(df_dt_120, how='left', on='ZIP')
print(df_labor_120.head(100))
print(df_labor_120.info())

      ZIP  ... sdzi_120_intersect_area
0     151  ...                0.147009
1     151  ...              116.482820
2     151  ...              116.482820
3     151  ...                0.139703
4     151  ...                0.612642
..    ...  ...                     ...
95  98101  ...                0.519979
96  98101  ...                0.519979
97  98101  ...                0.519979
98  98101  ...                0.519979
99  98101  ...                0.519979

[100 rows x 28 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12275823 entries, 0 to 12275822
Data columns (total 28 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   ZIP                                  int64  
 1   ZIP_Name                             object 
 2   2019 Jobs                            object 
 3   2020 Jobs                            object 
 4   2019 - 2020 Change                   object 
 5   Regional Completions (2019)          

## Calculating Total Net Commuters In Each Distance Range

### 15 min

## Labor Profiles 30 Min From Airports

In [ ]:
# df_cargo_30 = df_cargo.merge(df_dt_30, how='inner', on='ZIP')
# print(df_cargo_30.info())
# print(df_cargo_30.head(100))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3120 entries, 0 to 3119
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ZIP                         3120 non-null   int64  
 1   airport_name                3120 non-null   object 
 2   IATA_code                   3120 non-null   object 
 3   city                        3120 non-null   object 
 4   state                       3120 non-null   object 
 5   2019                        3120 non-null   object 
 6   2018                        3120 non-null   object 
 7   2017                        3120 non-null   object 
 8   2016                        3120 non-null   object 
 9   2015                        3120 non-null   object 
 10  2014                        3120 non-null   object 
 11  2013                        3120 non-null   object 
 12  2012                        3120 non-null   object 
 13  2011                        3120 

In [ ]:
# df_cargo_30.set_index(['ZIP', 'IATA_code'], inplace=True)
# df_cargo_30.head(100)

airport_name  ... sdzi_30_intersect_area
ZIP   IATA_code                                 ...                       
38116 MEM        Memphis International Airport  ...              10.428421
      MEM        Memphis International Airport  ...              17.337608
      MEM        Memphis International Airport  ...              17.337608
      MEM        Memphis International Airport  ...              17.337608
      MEM        Memphis International Airport  ...              17.337608
...                                        ...  ...                    ...
33166 MIA          Miami International Airport  ...               9.817293
      MIA          Miami International Airport  ...               9.817293
      MIA          Miami International Airport  ...               9.817293
      MIA          Miami International Airport  ...               9.817293
      MIA          Miami International Airport  ...               9.817293

[100 rows x 28 columns]

In [ ]:
# df_labor_cargo_30 = df_map_30_zc.reset_index().merge(df_cargo_30.reset_index(), 
#                                                      on='ZIP', how='inner').set_index([''])
# df_labor_cargo_30.info()
df_labor_cargo_30 = df_cargo.merge(df_map_30_zc.reset_index(), left_on='ZIP',
                                   right_on='sdzi_zip_cd', how='inner')
df_labor_cargo_30.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3093 entries, 0 to 3092
Data columns (total 52 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   airport_name                         3093 non-null   object 
 1   IATA_code                            3093 non-null   object 
 2   city                                 3093 non-null   object 
 3   state                                3093 non-null   object 
 4   2019                                 3093 non-null   object 
 5   2018                                 3093 non-null   object 
 6   2017                                 3093 non-null   object 
 7   2016                                 3093 non-null   object 
 8   2015                                 3093 non-null   object 
 9   2014                                 3093 non-null   object 
 10  2013                                 3093 non-null   object 
 11  2012                          

In [ ]:
df_labor_cargo_30.reset_index(inplace=True)
df_labor_cargo_30.head(100)

,IATA_code,sdzi_zip_cd,airport_name,city,state,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,10_year_per_chg,14_'19_per_chg,10_year_rank,14_'19_rank,address,ZIP_x,latitude,longitude,ZIP_y,ZIP_Name,2019 Jobs,2020 Jobs,2019 - 2020 Change,Regional Completions (2019),Automation Index,2020 Resident Workers,2020 Net Commuters,Avg. Hourly Earnings,Median Hourly Earnings,Median Annual Earnings,COL Index,COL Adjusted Avg. Hourly Earnings,COL Adjusted Median Hourly Earnings,COL Adjusted Median Annual Earnings,Occ. Mix Effect,Nat'l Growth Effect,Expected Change,Competitive Effect,2014 Median Hourly Earnings,2015 Avg. Hourly Earnings,sdzi_zip_nm,sdzi_longitude,sdzi_latitude,sdzi_30_intersect_zip_area,sdzi_30_intersect_area
0,MEM,38116.0,Memphis International Airport,Memphis,TN,"24,344,504,836","24,432,753,510","23,949,525,780","23,866,469,898","22,679,195,919","22,774,592,279","21,891,425,638","20,983,699,672","20,303,149,106","19,544,635,833","18,928,729,202","19,500,093,674",24.6%,6.9%,19,30,2491 Winchester Rd,38116,35.040031,-89.981873,38017,Collierville,13122,13068,-54,0.0,109.2,11514,1554.24,13.28,11.39,23681.63,96.00,13.83,11.86,24668.36,-20.96,44.24,23.28,-77.02,10.17,12.47,Memphis,-90.012806,35.031473,82.512026,0.000097
1,MEM,38116.0,Memphis International Airport,Memphis,TN,"24,344,504,836","24,432,753,510","23,949,525,780","23,866,469,898","22,679,195,919","22,774,592,279","21,891,425,638","20,983,699,672","20,303,149,106","19,544,635,833","18,928,729,202","19,500,093,674",24.6%,6.9%,19,30,2491 Winchester Rd,38116,35.040031,-89.981873,38103,Memphis,10474,10480,7,131.0,109.2,2554,7926.32,12.64,11.08,23038.48,96.00,13.16,11.54,23998.42,-16.73,35.31,18.58,-12.02,9.76,11.57,Memphis,-90.012806,35.031473,7.321858,3.889851
2,MEM,38116.0,Memphis International Airport,Memphis,TN,"24,344,504,836","24,432,753,510","23,949,525,780","23,866,469,898","22,679,195,919","22,774,592,279","21,891,425,638","20,983,699,672","20,303,149,106","19,544,635,833","18,928,729,202","19,500,093,674",24.6%,6.9%,19,30,2491 Winchester Rd,38116,35.040031,-89.981873,38104,Memphis,8103,8102,-2,86.0,109.2,5187,2914.31,12.65,11.07,23026.37,96.00,13.18,11.53,23985.80,-12.94,27.32,14.38,-16.20,9.79,11.67,Memphis,-90.012806,35.031473,4.904836,4.904836
3,MEM,38116.0,Memphis International Airport,Memphis,TN,"24,344,504,836","24,432,753,510","23,949,525,780","23,866,469,898","22,679,195,919","22,774,592,279","21,891,425,638","20,983,699,672","20,303,149,106","19,544,635,833","18,928,729,202","19,500,093,674",24.6%,6.9%,19,30,2491 Winchester Rd,38116,35.040031,-89.981873,38105,Memphis,1271,1274,3,164.0,109.2,1171,102.38,13.33,11.50,23922.28,96.00,13.89,11.98,24919.04,-2.03,4.29,2.26,0.28,9.96,12.01,Memphis,-90.012806,35.031473,1.775869,1.775869
4,MEM,38116.0,Memphis International Airport,Memphis,TN,"24,344,504,836","24,432,753,510","23,949,525,780","23,866,469,898","22,679,195,919","22,774,592,279","21,891,425,638","20,983,699,672","20,303,149,106","19,544,635,833","18,928,729,202","19,500,093,674",24.6%,6.9%,19,30,2491 Winchester Rd,38116,35.040031,-89.981873,38106,Memphis,3909,3919,10,0.0,109.2,4596,-677.61,14.28,12.93,26895.33,96.00,14.87,13.47,28015.97,-6.24,13.18,6.94,2.85,11.42,13.07,Memphis,-90.012806,35.031473,14.561133,12.206865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SDF,40209.0,Louisville International Airport,Louisville,KY,"15,599,137,404","14,643,310,355","13,403,682,652","12,804,148,855","12,057,543,654","11,568,369,154","21,891,425,638","20,983,699,672","20,303,149,106","19,544,635,833","18,928,729,202","19,500,093,674",-20.2%,34.8%,29,17,600 Terminal Dr,40209,38.175662,-85.736923,40242,Louisville,525,525,1,0.0,109.2,2217,-1691.05,13.84,12.47,25940.85,100.40,13.79,12.42,25837.50,-0.84,1.77,0.93,-0.21,10.46,12.13,Louisville,-85.748197,38.189486,3.367384,3.367384
96,SDF,40209

In [ ]:
df_30_grouped = df_labor_cargo_30.groupby(['IATA_code'])[["2020 Net Commuters", "2020 Jobs"]].sum()
df_30_grouped.head(48)

,2020 Net Commuters,2020 Jobs
IATA_code,,
ABE,533.58,158554
ABQ,9934.00,151017
AFW,37491.22,556352
ANC,8796.67,54036
ATL,60791.66,420673
AUS,38918.19,289275
BDL,28102.21,236505
BFI,134847.24,483693
BOS,108648.27,508978


In [ ]:
df_30_grouped_locations = df_cargo.merge(df_30_grouped.reset_index(), how='inner', on='IATA_code')
df_30_grouped_locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 46
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   airport_name        47 non-null     object 
 1   IATA_code           47 non-null     object 
 2   city                47 non-null     object 
 3   state               47 non-null     object 
 4   2019                47 non-null     object 
 5   2018                47 non-null     object 
 6   2017                47 non-null     object 
 7   2016                47 non-null     object 
 8   2015                47 non-null     object 
 9   2014                47 non-null     object 
 10  2013                47 non-null     object 
 11  2012                47 non-null     object 
 12  2011                47 non-null     object 
 13  2010                47 non-null     object 
 14  2009                47 non-null     object 
 15  2008                47 non-null     object 
 16  10_year_pe

In [ ]:
df_30_grouped_locations.drop(labels=['2019', '2018','2017','2016','2015','2014',
                                     '2013','2012','2011', '2010','2009','2008',
                                     '10_year_per_chg',"14_'19_per_chg"], axis=1,
                             inplace=True)
df_30_grouped_locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 0 to 46
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   airport_name        47 non-null     object 
 1   IATA_code           47 non-null     object 
 2   city                47 non-null     object 
 3   state               47 non-null     object 
 4   10_year_rank        47 non-null     int64  
 5   14_'19_rank         47 non-null     int64  
 6   address             47 non-null     object 
 7   ZIP                 47 non-null     int64  
 8   latitude            47 non-null     float64
 9   longitude           47 non-null     float64
 10  2020 Net Commuters  47 non-null     float64
 11  2020 Jobs           47 non-null     int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 4.8+ KB


In [ ]:
df_30_grouped_locations.to_excel('port_labor_30_min.xlsx', sheet_name='Sheet1')

In [ ]:
#df_labor_cargo_30.set_index(['IATA_code', 'ZIP_y'], inplace=True)
#df_labor_cargo_30.head(100)

sdzi_zip_cd  ... sdzi_30_intersect_area
IATA_code ZIP_y               ...                       
MEM       38017      38116.0  ...               0.000097
          38103      38116.0  ...               3.889851
          38104      38116.0  ...               4.904836
          38105      38116.0  ...               1.775869
          38106      38116.0  ...              12.206865
...                      ...  ...                    ...
SDF       40242      40209.0  ...               3.367384
          40243      40209.0  ...               4.094224
          40245      40209.0  ...              13.589036
          40258      40209.0  ...              11.553976
          40272      40209.0  ...              30.741593

[100 rows x 50 columns]

In [ ]:
df_labor_cargo_30.sort_index(inplace=True)
df_labor_cargo_30.head(100)

sdzi_zip_cd  ... sdzi_30_intersect_area
IATA_code ZIP_y               ...                       
ABE       8865       18109.0  ...               1.838891
          18011      18109.0  ...               0.027041
          18014      18109.0  ...              25.429896
          18015      18109.0  ...              21.826864
          18017      18109.0  ...              15.830236
...                      ...  ...                    ...
AFW       76017      76177.0  ...               1.098520
          76020      76177.0  ...               0.050474
          76021      76177.0  ...               7.396806
          76022      76177.0  ...               2.613423
          76023      76177.0  ...               0.434276

[100 rows x 50 columns]